# Purpose of this notebook
An obstacle for getting started with Neuroglancer is the data formats you can feed it are somewhat unfamiliar. For example, it does not accept TIFF format. It does this for a good reason though which is to be more efficient. One of the formats it accepts is called "precomputed" format, and that is the one we will use in this notebook. Fortunately there is a python pipeline for making precomputed data from TIFF files. 

Neuroglancer also is not set up to read CSV files. Instead, it uses the JSON file format, which essentially look like python dictionaries.

This notebook covers how to convert a custom annotated atlas volume (of the same format as the 'WHS_SD_rat_atlas_v3_annotation.tif' file on bucket) to precomputed format so that you can load it in to Neuroglancer. It then covers how to make a 3d mesh from this volume so that the atlas can be viewed in the 3d viewer within Neuroglancer. It also covers how to convert a CSV file formatted like the 'labels_v3.csv' file on bucket containing the region name mapping for the Rat MRI atlas into JSON format so that Neuroglancer can read it and display the region names.

In the following I use the original MRI atlas annotation volume and CSV file as an example. To use custom annotation volume and/or custom CSV file, replace the variables at the top of the notebook with your custom ones. The rest of the notebook should not (hopefully!) need to be changed before running.

## A quick note about Neuroglancer
Neuroglancer loads in datasets in "layers". A layer can be of type "image" (like what you would get as output from the light sheet microscope) or type "segmentation" (like an atlas annotation volume). The naming is a little confusing because both layer types refer to volumes (3-d objects). In this notebook, we are only concerned with a single layer: the annotation volume, which is a segmentation layer. If you were to make multiple annotation volumes (with different boundaries, etc.), each one of those would be a different layer. In Neuroglancer, you can overlay multiple layers or view them side-by-side. 

# Setup
In order to run the code in this notebook, you will need a conda environment with python3 and containing some additional libraries. This environment "ng_mriatlas" can be set up in the following way:
In terminal:
- conda create -n ng_mriatlas python=3.7.4 -y
- conda activate ng_mriatlas # (or source activate ng_mriatlas, depending on which version of conda you have)
- pip install cloud-volume
- pip install SimpleITK
- **pip install neuroglancer==2.8** <br>

\# Optional: if you want 3d meshing following the next steps
- git clone https://github.com/seung-lab/igneous.git igneous
- cd igneous
- pip install -r requirements.txt 
- python setup.py develop

\# To enable you to use jupyter notebooks to work with this environment as a kernel:
- pip install --user ipykernel
- python -m ipykernel install --user --name=ng_mriatlas

Once this is all installed, make sure to select this conda environment as the kernel when running this notebook (you might have to restart the notebook server)

In [12]:
import os,csv,json
import numpy as np
from cloudvolume import CloudVolume
from cloudvolume.lib import mkdir, touch
import SimpleITK as sitk

from concurrent.futures import ProcessPoolExecutor

import neuroglancer
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc

# Point to the custom annotation volume file that you have modified (in this example we point to the original one) 
custom_annotation_vol_path = 'WHS_SD_rat_atlas_v3_annotation.tif'

# Point to the csv file mapping region id to region name that you have modified (in this example we point to the original one)
annotation_csv_file = 'labels_v3.csv'

# Decide on a folder name where your layer for this volume is going to live. 
# IMPORTANT: For each custom atlas you need to change this so you don't 
# overwrite an old layer. You need the full path here. 
# In this example my layer folder will be 'my_custom_atlas_3d' saved in the current working directory
cwd = os.getcwd()
layer_dir = os.path.join(cwd,'my_custom_atlas_3d_v2')
# Make the layer directory (mkdir won't overwrite)
mkdir(layer_dir)
print(f"created {layer_dir}")
# Make the segment_properties directory as well - needed for our atlas labels later
segment_props_dir = os.path.join(layer_dir,'segment_properties')
mkdir(segment_props_dir)
print(f"created {segment_props_dir}")

    
# Finally, decide how many cpus you are willing and able to use for the parallelized conversion (see step 3)
cpus_to_use = 8

created /home/ahoag/progs/brodylab/atlas/my_custom_atlas_3d_v2
created /home/ahoag/progs/brodylab/atlas/my_custom_atlas_3d_v2/segment_properties


## Step 1: Convert the CSV annotation file into JSON file that Neuroglancer can read

This is an example to illustrate what the format of the final dictionary needs to look like:
```python
{"@type": "neuroglancer_segment_properties",
    "inline": {"ids": ["149"],
               "properties":[{"id": "label", "type": "label", "values":
                              ["PVT: Paraventricular nucleus of the thalamus"]}]}}
```
We can use this as a skeleton for our example. The only things we need to change are the "ids" and "properties" to use our ids and our labels

In [3]:
# First let's load our annotations from csv file into a simple dictionary
with open(annotation_csv_file,'r') as infile:
    reader = csv.reader(infile)
    next(reader) # skips the header since we just want the data and we already know what the order of the columns is
    annotation_dict = {row[2]:row[1] for row in reader}

In [4]:
# let's look at it to make sure it worked
annotation_dict

{'0': 'Clear Label',
 '1': 'descending corticofugal pathways',
 '2': 'substantia nigra',
 '3': 'subthalamic nucleus',
 '4': 'molecular layer of the cerebellum',
 '5': 'granule cell level of the cerebellum',
 '6': 'alveus of the hippocampus',
 '7': 'inferior cerebellar peduncle',
 '10': 'cingulate cortex, area 2',
 '30': 'striatum',
 '31': 'globus pallidus',
 '32': 'entopeduncular nucleus',
 '33': 'ventricular system',
 '34': 'medial lemniscus',
 '35': 'facial nerve',
 '36': 'anterior commissure, anterior part',
 '37': 'anterior commissure, posterior part',
 '38': 'ventral hippocampal commissure',
 '39': 'thalamus',
 '40': 'septal region',
 '41': 'optic nerve',
 '42': 'optic tract and optic chiasm',
 '43': 'pineal gland',
 '45': 'spinal cord',
 '46': 'commissure of the superior colliculus',
 '47': 'brainstem',
 '48': 'hypothalamic region',
 '50': 'superficial gray layer of the superior colliculus',
 '51': 'periaqueductal gray',
 '52': 'fornix',
 '53': 'mammillothalamic tract',
 '54': 'c

So we see that the IDs are the keys and labels are the values of this dictionary. Now let's make the segment properties dictionary like the skeleton above but now using our proper ids and labels 

In [14]:
segment_properties_dict = {"@type": "neuroglancer_segment_properties",
    "inline": {"ids": list(annotation_dict.keys()),
               "properties":[{"id": "label", "type": "label", "values":
                              list(annotation_dict.values())}]}}

In [15]:
# inspect it to make sure it worked
segment_properties_dict

{'@type': 'neuroglancer_segment_properties',
 'inline': {'ids': ['0',
   '1',
   '2',
   '3',
   '4',
   '5',
   '6',
   '7',
   '10',
   '30',
   '31',
   '32',
   '33',
   '34',
   '35',
   '36',
   '37',
   '38',
   '39',
   '40',
   '41',
   '42',
   '43',
   '45',
   '46',
   '47',
   '48',
   '50',
   '51',
   '52',
   '53',
   '54',
   '55',
   '56',
   '57',
   '58',
   '59',
   '60',
   '61',
   '62',
   '63',
   '64',
   '65',
   '66',
   '67',
   '68',
   '69',
   '70',
   '71',
   '72',
   '73',
   '74',
   '75',
   '76',
   '77',
   '78',
   '79',
   '80',
   '81',
   '82',
   '83',
   '84',
   '85',
   '92',
   '93',
   '94',
   '95',
   '96',
   '97',
   '98',
   '99',
   '100',
   '108',
   '109',
   '110',
   '112',
   '113',
   '114',
   '115',
   '119',
   '120',
   '121',
   '122',
   '123',
   '125',
   '126',
   '127',
   '128',
   '129',
   '130',
   '131',
   '132',
   '133',
   '134',
   '135',
   '136',
   '137',
   '138',
   '139',
   '140',
   '141',
   '142

Now all we need to do is save this as a file called "info" in a subfolder of our main layer directory called "segment_properties"

In [17]:
filename = os.path.join(layer_dir,"segment_properties","info")
with open(filename,'w') as outfile:
    json.dump(segment_properties_dict,outfile,indent=2)
print(f"Saved {filename}")

Saved /home/ahoag/progs/brodylab/atlas/my_custom_atlas_3d_v2/segment_properties/info


In [ ]:
# Feel free to inspect that file yourself to make sure it looks like the segment_properties_dict above

## Step 2: Write the instructions ("info") file that will tell Neuroglancer about your annotation volume and the annotation label file you just created
In the last step we created an info file inside the "segment_properties" subdirectory of our layer. Now we need to create an info file in the layer directory itself. This info file is also a JSON file, but it contains more general things about our layer like the shape and physical resolution of the volume. It will also have an instruction on where to find our segment properties info file that we just made. 

In [20]:
def make_info_file(resolution_xyz,volume_size_xyz,layer_dir):
    """ Make an JSON-formatted file called the "info" file
    for use with the precomputed data format. 
    Precomputed is one of the formats that Neuroglancer can read in.  
    --- parameters ---
    resolution_xyz:      A tuple representing the size of the pixels (dx,dy,dz) 
                         in nanometers, e.g. (20000,20000,5000) for 20 micron x 20 micron x 5 micron
    
    volume_size_xyz:     A tuple representing the number of pixels in each dimension (Nx,Ny,Nz)

                         
    layer_dir:           The directory where the precomputed data will be
                         saved
    """
    info = CloudVolume.create_new_info(
        num_channels = 1,
        layer_type = 'segmentation', # 'image' or 'segmentation'
        data_type = 'uint16', # 32-bit not necessary for atlases unless you have > 2^(32)-1 labels. Use smallest possible  
        encoding = 'raw', # other options: 'jpeg', 'compressed_segmentation' (req. uint32 or uint64)
        resolution = resolution_xyz, # X,Y,Z values in nanometers, 40 microns in each dim
        voxel_offset = [ 0, 0, 0 ], # values X,Y,Z values in voxels
        chunk_size = [ 1024, 1024, 1 ], # rechunk of image X,Y,Z in voxels.
        volume_size = volume_size_xyz, # X,Y,Z size in voxels
    )

    vol = CloudVolume(f'file://{layer_dir}', info=info)
    vol.provenance.description = "A test info file" # can change this if you want a description
    vol.provenance.owners = [''] # list of contact email addresses
    # Saves the info and provenance files for the first time
    vol.commit_info() # generates file://bucket/dataset/layer/info json file
    vol.commit_provenance() # generates file://bucket/dataset/layer/provenance json file
    # add a key for the segment properties that points to the directory that holds the segment properties info file
    info_dict = vol.info
    info_dict['segment_properties'] = "segment_properties"
    info_filename = '/'.join(vol.info_cloudpath.split('/')[2:]) 
    with open(info_filename,'w') as outfile:
        json.dump(info_dict,outfile,sort_keys=True,indent=2)
    print("Created CloudVolume info file: ",vol.info_cloudpath)

    return vol

In [21]:
## Make the info file

# The MRI atlas has 39 micron isotropic resolution - we need nanometers though
resolution_xyz = (39000,39000,39000)
# Load the MRI annotation volume into memory and get its shape 
annotation_vol = np.array(sitk.GetArrayFromImage(
    sitk.ReadImage(custom_annotation_vol_path)),dtype=np.uint16,order='F')
z_dim,y_dim,x_dim = annotation_vol.shape
volume_size_xyz = (x_dim,y_dim,z_dim)

# Write the info file
vol = make_info_file(
    resolution_xyz=resolution_xyz,
    volume_size_xyz=volume_size_xyz,
    layer_dir=layer_dir)


Created CloudVolume info file:  file:///home/ahoag/progs/brodylab/atlas/my_custom_atlas_3d_v2/info


# Step 3: Convert annotation volume to precomputed data format
First we create a directory (the "progress_dir") at the same folder level as the layer directory to keep track of the progress of the conversion. 
All the conversion does is copy the numpy array representing the 3d volume to a new object "vol". This is done one plane at a time (although it is parallelized). As each plane is converted, an empty file is created in the progress_dir with the name of the plane. By the end of the conversion, there should be as many files in this progress_dir as there are z planes. 

In [22]:
layer_name = layer_dir.split('/')[-1]
parent_dir = '/'.join(layer_dir.split('/')[:-1])
progress_dir = mkdir(parent_dir+ f'/progress_{layer_name}') # unlike os.mkdir doesn't crash on prexisting 
print(f"created directory: {progress_dir}")

created directory: /home/ahoag/progs/brodylab/atlas/progress_my_custom_atlas_3d_v2


In [23]:
def process_slice(z):
    """ This function copies a 2d image slice from the atlas volume
    to the cloudvolume object, vol. We will run this in parallel over 
    all z planes
    ---parameters---
    z:    An integer representing the 0-indexed z plane to be converted
    """
    if os.path.exists(os.path.join(progress_dir, str(z))):
        print(f"Slice {z} already processed, skipping ")
        return
    if z >= z_dim: # z is zero indexed and runs from 0-(z_dim-1)
        print("Index {z} >= z_dim of volume, skipping")
        return
    print('Processing slice z=',z)
    array = annotation_vol[z].reshape((1,y_dim,x_dim)).T
    vol[:,:, z] = array
    touch(os.path.join(progress_dir, str(z)))
    return "success"


In [24]:
# Run the conversion in parallel. It's not a huge amount of processing but the more cores the better

# First figure out if there are any planes that have already been converted 
# by checking the progress dir
done_files = set([ int(z) for z in os.listdir(progress_dir) ])
all_files = set(range(vol.bounds.minpt.z, vol.bounds.maxpt.z))
# Figure out the ones we still need to convert 
to_upload = [ int(z) for z in list(all_files.difference(done_files)) ]
to_upload.sort()
print(f"Have {len(to_upload)} planes to upload")
with ProcessPoolExecutor(max_workers=cpus_to_use) as executor:
    for result in executor.map(process_slice,to_upload):
        try:
            print(result)
        except Exception as exc:
            print(f'generated an exception: {exc}')

Have 455 planes to upload
Processing slice z= 1
Processing slice z= 0
Processing slice z= 3
Processing slice z= 2
Processing slice z= 6
Processing slice z= 7
Processing slice z= 5
Processing slice z= 4


Uploading: 100%|██████████| 1/1 [00:00<00:00, 66.58it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00, 90.56it/s]

Processing slice z= 8



Uploading: 100%|██████████| 1/1 [00:00<00:00, 15.70it/s]

Processing slice z= 9


...

Processing slice z= 453


Uploading: 100%|██████████| 1/1 [00:00<00:00, 15.23it/s]


Processing slice z= 454


Uploading: 100%|██████████| 1/1 [00:00<00:00, 16.53it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00, 24.11it/s]


success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


## Step 4 (optional): Make the 3d mesh
In this step, we make the 3d mesh of the atlas from the precomputed layer we just created. This step is necessary if you want to view the atlas you created in the 3d viewer (lower left panel in Neuroglancer).  

In [6]:
# define the meshing function
def make_3d_mesh(vol,n_cores):
    """ This function makes the mesh so that when you
    load your segmentation layer into neuroglancer, you 
    will be able to see whichever segments are highlighted
    in the 3d viewer as well as all of the other panels. 

    It uses parallel processing to speed up the meshing
    ---parameters---
    vol:     The cloudvolume object
    n_cores: Number of cores to use
    """
    # Mesh using the  cores, use True to use all cores
    cloudpath = vol.cloudpath
    with LocalTaskQueue(parallel=n_cores) as tq:
        tasks = tc.create_meshing_tasks(cloudpath, mip=0, shape=(256, 256, 256))
        tq.insert_all(tasks)
        tasks = tc.create_mesh_manifest_tasks(cloudpath)
        tq.insert_all(tasks)
    print("Done!")


In [7]:
# call the meshing function (should take <~ 1 minute with 8 or more cores)
make_3d_mesh(vol=vol,n_cores=cpus_to_use)

Downloading: 100%|██████████| 257/257 [00:01<00:00, 231.07it/s][A

Downloading: 100%|██████████| 257/257 [00:01<00:00, 223.19it/s][A

Downloading: 100%|██████████| 257/257 [00:01<00:00, 211.96it/s][A

Downloading:  51%|█████▏    | 132/257 [00:01<00:11, 11.27it/s]


Downloading: 100%|██████████| 257/257 [00:01<00:00, 183.15it/s][A

Downloading: 100%|██████████| 257/257 [00:01<00:00, 170.06it/s][A

Downloading: 100%|██████████| 257/257 [00:01<00:00, 157.78it/s][A

Downloading: 100%|██████████| 257/257 [00:01<00:00, 151.26it/s][A

Downloading: 100%|██████████| 257/257 [00:01<00:00, 148.60it/s][A

Downloading: 100%|██████████| 199/199 [00:00<00:00, 355.62it/s]

Downloading: 100%|██████████| 199/199 [00:00<00:00, 326.11it/s]

Downloading: 100%|██████████| 199/199 [00:00<00:00, 323.41it/s]

Downloading: 100%|██████████| 199/199 [00:00<00:00, 385.52it/s]

Downloading:  54%|█████▍    | 107/199 [00:00<00:00, 292.61it/s]


Downloading: 100%|██████████| 199/199 [00:00<00:00, 385.97it/s]

Downloading: 100%|██████████| 199/199 [00:00<00:00, 384.32it/s]

Tasks: 100%|██████████| 999/999 [00:03<00:00, 291.29it/s]
Tasks: 0it [00:00, ?it/s]

Done!


If that worked, then it will autom

# Step 5: Host the precomputed data on your machine so that Neuroglancer can see it
This step is really easy! Note: Exectuing the code below will cause your jupyter notebook to hang, so it is better to run the following code in a new ipython terminal (make sure to have the ng_mriatlas conda environment activated in that python session) rather than the notebook. 

```python
from cloudvolume import CloudVolume
vol = CloudVolume(f'file://{layer_dir}')
vol.viewer(port=1338)
```

# Step 6: View your custom volume and labels in Neuroglancer
Step 4 hosts your data via http on port 1338 of your local machine. To actually view your data in Neuroglancer, there are two ways to do this. You can either load the data in manually in the browser or load it in with python. 

For the manual method, open up the Braincogs Neuroglancer client: [https://nglancer.pni.princeton.edu](https://nglancer.pni.princeton.edu) (you must be using a Princeton VPN) and then click the "+" in the upper left hand corner of the screen once the black screen loads. To load in your data, type the following into the source text box:<br>
> precomputed://http://localhost:1338 <br>

Then hit tab and name your layer if you'd like. Hit enter or the "add layer" button and your layer should load into Neuroglancer. Hopefully the labels you added should be showing up in the bottom left when you hover over a region. 

For the python method, you can do this by executing the following cells. Make sure you have hosted the data in another python instance somewhere on your local machine at port 1338.

In [25]:
# Set which client you want to use - use the BRAINCOGS client to get the latest features.
# Need to be in the Princeton VPN to use this
neuroglancer.set_static_content_source(url='https://nglancer.pni.princeton.edu')

In [26]:
# Make a viewer object that represents your connection to the Neuroglancer session
viewer = neuroglancer.Viewer()

In [27]:
# Load in the layer with name "my_custom_atlas_3d" or call it whatever you want. 
# This is the name for the layer that appears in the box in the upper left of the screen
# when you load in the layer.

# This cell generates a link, which when clicked brings you to the
# Neuroglancer browser interface with your data loaded in 
with viewer.txn() as s:
    s.layers['atlas'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1338',
    )
print(viewer)

http://127.0.0.1:39868/v/327ae353b0aa47db2e4db1b9de250e685604b9d7/


Once you're in Neuroglancer, when you hover over a segment the region name shows up at the top and no longer the bottom. You can either select individual segments by double clicking on them or using the panel on the right. Activate the right panel by right clickin gon the the layer name in the upper left to get a list of the segments and their region names. Select a checkbox to select a segment in the viewer. 